In [47]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chosen-file-name/pdf_filenames.csv
/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json


In [36]:
pip install langdetect


ERROR! Session/line number was not unique in database. History logging moved to new session 29
Note: you may need to restart the kernel to use updated packages.


In [48]:
import json
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from tqdm import tqdm
import csv

In [89]:
input_path = '/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json'
output_path = '/kaggle/working/english_arxiv_full_cleaned_final6.csv'
pdf_filenames_csv = '/kaggle/input/chosen-file-name/pdf_filenames.csv'  # Replace this with your CSV file path

In [90]:
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False


In [91]:
def extract_latest_version_info(versions):
    if not versions or not isinstance(versions, list):
        return "", ""
    latest = versions[-1]
    return latest.get('version', ''), latest.get('created', '')


In [92]:
def flatten_authors(authors_parsed):
    if not authors_parsed or not isinstance(authors_parsed, list):
        return ""
    return ", ".join(" ".join(filter(None, author)) for author in authors_parsed)

In [93]:
pdf_filenames_set = set()
with open(pdf_filenames_csv, 'r', newline='', encoding='utf-8') as pdf_file:
    pdf_reader = csv.reader(pdf_file)
    next(pdf_reader)  # Skip header
    for row in pdf_reader:
        if row:
            pdf_filenames_set.add(row[0].strip())

In [94]:
print(len(pdf_filenames_set))

56655


In [95]:
total_lines = 2600000  # Estimate for tqdm (can adjust if needed)
english_count = 0
fieldnames_written = False

In [96]:
with open(input_path, 'r') as infile, open(output_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = None
    fieldnames_written = False
    english_count = 0

    progress = tqdm(infile, total=total_lines, desc=f"Stored: {english_count}")

    for line in progress:
        try:
            record = json.loads(line)
            text = f"{record.get('title', '')} {record.get('abstract', '')}".strip()
    
            # Get version info FIRST to construct the PDF filename
            latest_version, latest_created = extract_latest_version_info(record.get("versions", []))
            pdf_filename = f"{record['id']}{latest_version}.pdf"
    
            # Only keep if the file is in your allowed PDF names
            if pdf_filename not in pdf_filenames_set:
                continue
    
            # Now check if it's English
            if not text or not is_english(text):
                continue
    
            # ✅ Process the matching record
            record["latest_version"] = latest_version
            record["latest_created"] = latest_created
            record["pdf_filename"] = pdf_filename
            record.pop("versions", None)
    
            combined_authors = flatten_authors(record.get("authors_parsed", []))
            record["authors_combined"] = combined_authors
            record.pop("authors_parsed", None)
    
            if writer is None:
                fieldnames = list(record.keys())
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                fieldnames_written = True
    
            writer.writerow(record)
            english_count += 1

            # 🔄 Update progress bar description
            progress.set_description(f"Stored: {english_count}")

        except (json.JSONDecodeError, UnicodeEncodeError, KeyError):
            continue  # Skip broken lines or missing keys

print(f"\n✅ English records written: {english_count}")
print(f"📁 Saved cleaned data to: {output_path}")


Stored: 36346: : 2710806it [04:38, 9750.27it/s]                            


✅ English records written: 36346
📁 Saved cleaned data to: /kaggle/working/english_arxiv_full_cleaned_final6.csv
